In [309]:
import math
from web3 import Web3, AsyncWeb3
from web3 import eth

import abi as abi_import
import time
from datetime import datetime


In [312]:
w3 = Web3(Web3.HTTPProvider('https://eth-mainnet.g.alchemy.com/v2/HsRAh-rTKhWRIzJAVCfUU3VXFqilsfLf'))
w3.is_connected()

True

In [313]:
sy_addr = '0xED97f94dd94255637A054098604E0201C442a3FD'
sy_abi = abi_import.sy_token_abi

mkt_abi = abi_import.mkt
mkt_addr = '0xC9beCdbC62efb867cB52222b34c187fB170379C6'


In [314]:
contract_instance = w3.eth.contract(address=mkt_addr, abi=mkt_abi)

In [315]:
mkt_data = contract_instance.functions._storage().call()

In [316]:
total_pt = mkt_data[0]
total_sy = mkt_data[1]
impliedRate = mkt_data[2]

In [317]:
pt_eth =  float(Web3.from_wei(total_pt, 'ether'))
sy_eth =  float(Web3.from_wei(total_sy, 'ether'))

In [318]:
pt_ratio = pt_eth / (pt_eth + sy_eth)
px_ratio = pt_ratio / (1-pt_ratio)

In [319]:
ln_lastImpliedRate = float(Web3.from_wei(impliedRate, 'ether'))
lastImpliedRate = math.exp(ln_lastImpliedRate)

In [320]:
scalar_root =  contract_instance.functions.scalarRoot().call()

In [321]:
initAnchor = contract_instance.functions.initialAnchor().call()

In [322]:
initAnchor_eth = float(Web3.from_wei(initAnchor, 'ether'))
scalar_root_eth = float(Web3.from_wei(scalar_root, 'ether'))

In [323]:
startTime = datetime(2023, 10, 11, 3, 19, 11).timestamp()
startTime

1697008751.0

In [324]:
expiry = contract_instance.functions.expiry().call()
unixYear = 31556926

In [325]:
(expiry - startTime) / unixYear

0.4618716347720307

In [326]:
def update_now():
    return time.time()

def rateScalar(nowTime):
    return scalar_root_eth / (nowTime / unixYear)

def yrsLeft(nowTime):
    return (expiry - nowTime) / unixYear

def rateAnchor(nowTime):
    return lastImpliedRate**yrsLeft(nowTime) - (math.log(pt_ratio/(1-pt_ratio))) / rateScalar(nowTime)

In [327]:
rtScalar = rateScalar(update_now())
rtAnchor = rateAnchor(update_now())

In [328]:
exchange_rt_px = ((math.log(pt_ratio/(1-pt_ratio))) / rtScalar) + rtAnchor

In [329]:
pt_price = 1 / exchange_rt_px
sy_price = pt_price * ((lastImpliedRate)**(yrsLeft(update_now())) - 1)

In [344]:
silo =  '[{"inputs":[{"internalType":"contract ISiloRepository","name":"_siloRepo","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"DifferentArrayLength","type":"error"},{"inputs":[],"name":"InvalidRepository","type":"error"},{"inputs":[],"name":"UnsupportedLTVType","type":"error"},{"inputs":[],"name":"UserIsZero","type":"error"},{"inputs":[],"name":"ZeroAssets","type":"error"},{"inputs":[{"internalType":"uint256","name":"_assetTotalDeposits","type":"uint256"},{"internalType":"contract IShareToken","name":"_shareToken","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"balanceOfUnderlying","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"borrowAPY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"borrowShare","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"calcFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"calculateBorrowValue","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"calculateCollateralValue","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"collateralBalanceOfUnderlying","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"collateralOnlyDeposits","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"debtBalanceOfUnderlying","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"depositAPY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"},{"internalType":"uint256","name":"_timestamp","type":"uint256"}],"name":"getBorrowAmount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"},{"internalType":"uint256","name":"_timestamp","type":"uint256"}],"name":"getDepositAmount","outputs":[{"internalType":"uint256","name":"totalUserDeposits","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"getModel","outputs":[{"internalType":"contract IInterestRateModel","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"getUserLTV","outputs":[{"internalType":"uint256","name":"userLTV","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"getUserLiquidationThreshold","outputs":[{"internalType":"uint256","name":"liquidationThreshold","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"getUserMaximumLTV","outputs":[{"internalType":"uint256","name":"maximumLTV","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"getUtilization","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"hasPosition","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"inDebt","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"isSolvent","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"lensPing","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"liquidity","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"protocolFees","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"siloRepository","outputs":[{"internalType":"contract ISiloRepository","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalBorrowAmount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalBorrowAmountWithInterest","outputs":[{"internalType":"uint256","name":"_totalBorrowAmount","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalBorrowShare","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalDeposits","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalDepositsWithInterest","outputs":[{"internalType":"uint256","name":"_totalDeposits","type":"uint256"}],"stateMutability":"view","type":"function"}]'


silo_addr = '0x32a4Bcd8DEa5E18a12a50584682f8E4B77fFF2DF'

siloYield = '0x96eFdF95Cc47fe90e8f63D2f5Ef9FB8B180dAeB9'
yieldToken = '0xD533a949740bb3306d119CC777fa900bA034cd52'
silo_contract = w3.eth.contract(address=silo_addr, abi=silo)


In [346]:
borrowAPY = silo_contract.functions.borrowAPY(siloYield, yieldToken).call()

In [348]:
borrowAPY_eth = float(Web3.from_wei(borrowAPY, 'ether'))
borrowAPY_eth

0.069180134581872

In [352]:
depositAPY = silo_contract.functions.depositAPY(siloYield, yieldToken).call()
depositAPY_eth = float(Web3.from_wei(depositAPY, 'ether'))
depositAPY_eth

0.03418558324355485

In [355]:
util = silo_contract.functions.getUtilization(siloYield, yieldToken).call()
util_eth = float(Web3.from_wei(util,'ether'))
util_eth

0.49414381843903366

In [358]:
liq = silo_contract.functions.liquidity(siloYield, yieldToken).call()
liq_eth = float(Web3.from_wei(liq,'ether'))
liq_eth

4236322.310791544

In [359]:
borrow = silo_contract.functions.totalBorrowAmount(siloYield, yieldToken).call()
borrow_eth = float(Web3.from_wei(borrow,'ether'))
borrow_eth

4132055.192710716

In [360]:
deposits = silo_contract.functions.totalDeposits(siloYield, yieldToken).call()
deposits_eth = float(Web3.from_wei(deposits,'ether'))
deposits_eth

8362049.748519761